# Chapter 1: Single cell simulation with current injection (for Project S5)

In this example we will build a network consisting of a single biophysically detailed cell. Then we will run a short simulation with a current injection at the soma of the cell, then look at the output of spikes, membrane potential and calcium flux.


**Note** - This example is adapted from the __[BMTK Tutorial 01](https://github.com/AllenInstitute/bmtk/blob/develop/docs/tutorial/01_single_cell_clamped.ipynb)__.

**requirements:**
* Python 2.7, 3.6+
* bmtk
* NEURON 7.4+

## 1. Building the network

First step is to use the bmtk Network Builder to create and save the network. First we instantiate a network with a name or our choosing (since throughout this tutorial we will use cell models that from the mouse cortex, let's call our network 'mcortex'). 

Once we have a network, we can add a single node by calling the add_nodes method.

In [ ]:
from bmtk.builder.networks import NetworkBuilder

net = NetworkBuilder('Nr5a1-Cre')
net.add_nodes(cell_name='386049446',
              potental='exc',
              model_type='biophysical',
              model_template='ctdb:Biophys1.hoc',
              model_processing='aibs_perisomatic',
              dynamics_params='386049446_fit.json',
              morphology='Nr5a1-Cre_Ai14-177334.05.01.01_491459171_m.swc')

Some of the parameters used to create the node are optional and only for our benefit. Others are necessary for when we will eventually run a simulation:
* *cell_name* (optional) - Name/type of cell we will be modeling.
* *potential* (optional) - Use to indicate that it is an excitatory type cell.
* *model_type* - Used by the simulator to indicate that we are using a biophysical cell.
* __*dynamics_params*__ - Model parameters. File will be downloaded from the Allen Cell Types Database. 
* __*morphology*__ - Model morphology. File will be downloaded from the Allen Cell Types Database.
* *model_processing* - A custom function used by the simulator to load the model into NEURON using Allen Cell-Types files for perisomatic models ('aibs_allactive' for all_active model).
 
#### Building and saving
The final thing to do is to build and save the network. If successful, we should see a combination of hdf5 and csv files in the './network' directory, these files are used describe the network, and can be saved, stored and run at a later date.

First, it's a good idea to remove any old files in the "network" folder so they don't interfere with the current simulation.

In [ ]:
%%bash
rm network/*

If you get the output:

rm: cannot remove 'network/*': No such file or directory

It's OK. Keep going.

In [ ]:
net.build()
net.save_nodes(output_dir='network')

Use the NetworkBuilder nodes() method to show that a node of our parameters was created

In [ ]:
for node in net.nodes():
    print(node)

## 2. Setting up the simulator enviornment

Now that the network has been built, we can use the BioNet simulator to setup and run it using NEURON. The easiest ways to do this is to copy and modify [existing simulation setup](https://github.com/AllenInstitute/bmtk/tree/develop/docs/examples/bio_14cells), or use the  ```bmtk.utils.sim_setup``` script which can be called from the command-line

```bash
$ python -m bmtk.utils.sim_setup  --report-vars v,cai --network network --iclamp 0.18,500.0,1000.0 --dt 0.025 --tstop 2000.0 --include-examples --compile-mechanisms bionet Proj_S5
```
Or call the function directly in python

In [ ]:
from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(base_dir='Proj_S5',      # Where to save the scripts and config files 
                 network_dir='network',    # Location of directory containing network files
                 tstop=2000.0, dt=0.025,     # Run a simulation for 2000 ms at 0.025 ms intervals
                 report_vars=['v', 'cai'], # Tells simulator we want to record membrane potential and calcium traces
                 current_clamp={           # Creates a step current from 500.ms to 1500.0 ms  
                     'amp': 0.180,
                     'delay': 500.0,
                     'duration': 1000.0
                 },
                 include_examples=True,    # Copies components files
                 compile_mechanisms=True   # Will try to compile NEURON mechanisms
                )

<div class="alert alert-warning">
**Warning**: You may see an error that it filed while trying to compile the NEURON mechanisms - especially if jupyter is unable to find your NEURON environment path. If this happens you will need to manually compile the mechanisms:  

```
 cd Proj_S5/components/mechanisms  
 nrnivmodl modfiles
```
</div>



This will create the directory **Proj_S5** (feel free to give it a better name or set it to a different locations) with a number of new files and folders. Some of the more important ones include:
* **circuit_config.json** - A configuration file that contains the location of the network files we created above. Plus location of neuron and synpatic models, templates, morphologies and mechanisms required to build our instantiate individual cell models.


* **simulation_config.json** - contains information about the simulation. Including initial conditions and run-time configuration (_run_ and _conditions_). In the _inputs_ section we define what external sources we will use to drive the network (in this case a current clamp). And in the _reports_ section we define the variables (soma membrane potential and calcium) that will be recorded during the simulation 


* **components/biophysical_neuron_models/472363762_fit.json** - The parameter file for the cell we're modeling. Originally [downloaded from the Allen Cell Types Database](http://celltypes.brain-map.org/neuronal_model/download/482934212)


* **components/biophysical_neuron_models/Scnn1a_473845048_m.swc** - The morphology file for our cell. Originally [downloaded from the Allen Cell Types Database](http://celltypes.brain-map.org/neuronal_model/download/482934212)


Modifying these files with a text editor, or swapping out different parameter or morphologies, allows changing the simulation without extra programming.

### Place the model files in proper directory
We will need to move the parameters and morphology files we will be using in the model into our simulator enviornment by doing either
* Copy *386049446_fit.json*  and place it in /Proj_S5/components/biophysical_neuron_models
* Copy *Nr5a1-Cre_Ai14-177334.05.01.01_491459171_m.swc* and place it in /Proj_S5/components/morphologies

or directly run the command below in python.

In [ ]:
%%bash
cp source/386049446_fit.json Proj_S5/components/biophysical_neuron_models
cp source/Nr5a1-Cre_Ai14-177334.05.01.01_491459171_m.swc Proj_S5/components/morphologies

## 3. Running the simulation

Once our config file is setup we can run a simulation either through the command line:
```bash
$ python run_bionet.py simulation_config.json
```

or through the script

In [ ]:
from bmtk.simulator import bionet

conf = bionet.Config.from_json('Proj_S5/simulation_config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

<div class="alert alert-warning">
**Warning**: If you the the following error
    argument not a density mechanism name.
You will need to compile the NEURON mechanisms  


```
 cd Proj_S5/components/mechanisms
 nrnivmodl modfiles
 
```
</div>

A quick breakdown of the script:
```python
conf = config.from_json('config.json')
io.setup_output_dir(conf)
nrn.load_neuron_modules(conf)
```
This section loads the configuration file, it setups the output directory and files for writing during the simulation, and loads NEURON mechanisms needed by the cell model(s) during the simulation.

```python
net = bionet.BioNetwork.from_config(conf)
```
Creates a NEURON representation of the network, including cell models that have been converted into their NEURON equivelents.

```python
sim = Simulation.from_config(conf, network=net)
sim.run()
```
Sets up and runs the NEURON simulation. When finished the output - spike times, membrane potential and Calcium influx, will be saved into the output directory as specified in the config.



## 4. Analyzing the run

The results of the simulation are placed into various files as specified in the "output" section of the config file. We can change this before run-time if required.

All simulations will save the spike times of the network cells. These are saved in csv format (output/spikes.txt) or hdf5 format(output/spikes.h5). To get the table of spike times for our single-cell network we can run the following method from the analyzer (gid 0 corresponds to our single cell).

In [ ]:
from bmtk.analyzer.spike_trains import to_dataframe
to_dataframe(config_file='Proj_S5/simulation_config.json')

When setting up the enviornment and config file we specified cell_vars=['v', 'cai']. This indicates to the simulator to also record membrane potential and calcium diffusion (and we can also specify other variables as long as they are supported in NEURON).

In [ ]:
%matplotlib inline
from bmtk.analyzer.cell_vars import plot_report

plot_report(config_file='Proj_S5/simulation_config.json')

Following script allows you to get recorded variables directly from __.h5__ data file. You can further analyze the results if needed.  
__[Quick Start Guide](http://docs.h5py.org/en/stable/quick.html)__ for h5py data type.

In [ ]:
import h5py
import numpy as np

# get recorded variable from h5 file into numpy array
f = h5py.File('Proj_S5/output/v_report.h5','r') # load data file of membrane voltage recording
f.visit(print) # check groups

print(f['report/Nr5a1-Cre/mapping/time'][()]) # time (start, stop, step size)
t = np.arange(*f['report/Nr5a1-Cre/mapping/time']) # create a numpy array of time steps
v = f['report/Nr5a1-Cre/data'][()] # get membrane voltage data to a numpy array
node_id = f['report/Nr5a1-Cre/mapping/node_ids'][0] # node id (cell id is 0 since there is only 1 cell in the 'network')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# plot the membrane voltage
plt.figure()
plt.plot(t,v)
plt.xlabel('ms')
plt.ylabel('mV')
plt.title('v')
plt.legend(['node-id '+str(node_id)])
plt.show()

## 5. Additional Information

### Changing run-time parameters.
The input section lets us control stimulus onto the network. There are a number of different options which will be explained in the following tutorials. But even with a simple current injection we can adjust amplitude, delay and stimulation duration and measure the effect on the cell.
```json
"inputs": {
 	"current_clamp": {
      "input_type": "current_clamp",
      "module": "IClamp",
      "node_set": "all",
      "amp": 0.120,
      "delay": 500.0,
      "duration": 1000.0
    }
}
```

### Changing cell models
When building the network we defined the cell model and morphology through the 'dynamics_params' and 'morphology_file' options. After building and saving the network, these values were saved in the node-types csv file.

In [ ]:
import pandas as pd
pd.read_csv('network/Nr5a1-Cre_node_types.csv', sep=' ')

If we want to run the simulation on a different cell model, all we have to do is 
1. Download new parameters.json and morphology.swc into components/biophysical_neuron_models and components/morphologies respectively.
2. Open Nr5a1-Cre_node_types.csv in a text editor and update 'morphology_file' and 'params_file' accordingly.

In our simple one-cell example, it is likely faster to just rebuild the network. However the advantage of the use of the node types becomes clear once we start dealing with a larger network. For example we may have a network of hundreds of thousands of individual cells with tens of thousands of Scnn1a type cells. The process of adjusting/changing the Scnn1a parameter in the csv then starting another simulation only takes seconds, whereas rebuilding the entire network may take hours.